In [1]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from beakerx import *
plt.style.use('ggplot')
import seaborn as sns
sns.set()
import h5py
import os
from tqdm import tqdm_notebook as tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/home/xupeng/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 总结下面所做的工作.  
根据实际，在两个点的实际距离较近的时候，特征之间也应该相近<br>
relation函数统计两个点的给定特征的相似度，相似度越高，值越接近1。<br>
corelation函数统计对于给定的特征，这些特征是否足够优秀。理论上讲，优秀的特征应该在点的距离近的时候比较类似。所以点的距离越小，relation结果越大。因此，corelation越接近-1，说明特征越好。<br>
对于10个特征，20个特征，30个特征各测试一番。<br>
数据说明，各测试十万个，最终相关值储存在 rela_10 rela_20 rela_30中。<br>
标签储存在labe_10, labe_20, labe_30中<br>
其中用nr_10 nr_20 nr_30来表示计算过程中，有多大比率的nan值。

In [2]:
#index_1 index_2 should be two of the 1500 receiver
def relation(index_1, index_2, features):
    xdata=combine_data[index_1,features]
    ydata=combine_data[index_2,features]
    an=~np.isnan(xdata)&~np.isnan(ydata)
    nr=len(an)/len(xdata)
    xdata=xdata[an]
    ydata=ydata[an]
    r,_=scipy.stats.pearsonr(xdata,ydata)
    return r,nr

In [3]:
def corelation(features):
    testnum1=10
    testnum2=50
    r=np.zeros(testnum1)
    nr=np.zeros(testnum1)
    for j in range(testnum1):
        x=np.zeros(testnum2)
        y=np.zeros(testnum2)
        ur=np.zeros(testnum2)
        n=0
        xn=np.random.randint(1,1500,testnum2)
        yn=np.random.randint(1,1500,testnum2)
        #random choose 100 points to calculate the relation between distance and feature relation
        for i in range(testnum2):
            x[n]=distance[xn[i],yn[i]]
            y[n],ur[n]=relation(xn[i],yn[i],features)
            
            n=n+1
                #print(relation(i,j,features))
                #np.random.randint(1,100,[5,5])
        r[j],_=scipy.stats.pearsonr(x,y)
        nr[j]=np.mean(ur)
    an=~np.isnan(r)
    r=r[an]
    nr=nr[an]
    return np.mean(r), np.mean(nr)

In [4]:
rela_10=np.zeros(10000)
rela_20=np.zeros(10000)
rela_30=np.zeros(10000)
labe_10=np.random.randint(1,500,size=(10000,10))
labe_20=np.random.randint(1,500,size=(10000,20))
labe_30=np.random.randint(1,500,size=(10000,30))
nr_10=np.zeros(10000)
nr_20=np.zeros(10000)
nr_30=np.zeros(10000)

In [ ]:
for i in tqdm(range(10000)):
    rela_10[i],nr_10[i]=corelation(labe_10[i])
    rela_20[i],nr_20[i]=corelation(labe_20[i])
    rela_30[i],nr_30[i]=corelation(labe_30[i])

In [ ]:
import argparse

parser = argparse.ArgumentParser(description='test some correlation')
parser.add_argument('--c', dest='count',  default=10,type=int help='feature counts')
args = parser.parse_args()

feature_counts = args.count
